In [1]:
library(pacman)
p_load(tidyverse, plyr, sf, ggspatial, rnaturalearth, rnaturalearthdata,RColorBrewer, wesanderson)
crs_canalb <- "+proj=aea +lat_1=50 +lat_2=70 +lat_0=40 +lon_0=-96 +x_0=0 +y_0=0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs"
tcc_type = "ge_1p37m"
maindir = 'D:\\projects\\ABoVE_lidar'#'C:/Users/pmontesa/Google Drive/ABoVE_lidar'
pal_rev_spectral9 = rev(brewer.pal(9,"Spectral"))

In [2]:
lm_eqn = function(df){
      y=df[,1]
      x=df[,2]
      m = lm(y ~ x, df);

      eq <- substitute(italic(y) == a + b %.% italic(x)*","~~italic(r)^2~"="~r2,
           list(a = format(coef(m)[1], digits = 2),
                b = format(coef(m)[2], digits = 2),
               r2 = format(summary(m)$r.squared, digits = 3)))
      int = format(coef(m)[1], digits = 2)
      slope = format(coef(m)[2], digits = 2)
      # https://stackoverflow.com/questions/43123462/how-to-obtain-rmse-out-of-lm-result

      rss   <- c(crossprod(m$residuals))
      mse   <- rss / length(m$residuals)

      rmse  <- format(sqrt(mse),dig=2)
      sig2  <- format(rss / m$df.residual, dig=2) # Pearson estimated residual variance (as returned by summary.lm):
      n     <- nobs(m)
      r2    <- format(summary(m)$r.squared, dig = 2)
      mae   <- format(mean(abs(m$residuals)), dig = 2)
      mape  <- format(mean(abs((m$residuals)/y))*100, dig=2)  #https://stats.stackexchange.com/questions/299712/what-are-the-shortcomings-of-the-mean-absolute-percentage-error-mape

      #return(as.character(as.expression(eq)))
      return(data.frame(cbind(int, slope, r2, rmse, sig2, n, mae, mape)))
  }
val_plot_geom_list = list(
        ggtitle("Boreal tree canopy cover: LVIS vs GLiHT", subtitle = paste0("Tree canopy cover (",tcc_type,") by topographic aspect (Flat = < 3 slope)")),
        #scale_fill_gradientn( labels = my_breaks, breaks = my_breaks, limits = c(min(my_breaks),max(my_breaks)+3000),colours=pal_rev_spectral9, trans = "log", name='# obs.'),
        scale_fill_gradientn( labels = c(1,10,100), breaks = c(1,10,100), limits = c(1,500), colours=pal_rev_spectral9, trans = "log", name='# obs.'),
        guides(colour = guide_legend(override.aes = list(alpha = 1))),
        #geom_vline(xintercept = 0, linetype = "dashed"),
        #geom_hline(yintercept = 0, linetype = "dashed"),
        geom_abline(intercept = 0, slope = 1, size= 0.25),
        stat_smooth(method = "lm", col = "black", linetype = "dashed"),
        theme_minimal()+
          theme(axis.text = element_text(size = 6)),
        coord_fixed()
)

In [3]:
#
# TCC: Check LVIS tree canopy cover estimate with GLiHT
#
#


samples_val_gliht2014_lvis2017 = data.frame(read.csv(paste0(maindir, "/", "SAMPLE_stack_tcc_gliht2014_tcc_lvis2017.csv"), header=T))
samples_val_gliht2014_lvis2017$var_ref = "GLiHT 2014"
samples_val_gliht2014_lvis2017$var_src = "LVIS 2017"
samples_val_gliht2014_lvis2019 = data.frame(read.csv(paste0(maindir, "/", "SAMPLE_stack_tcc_gliht2014_tcc_lvis2019.csv"), header=T))
samples_val_gliht2014_lvis2019$var_ref = "GLiHT 2014"
samples_val_gliht2014_lvis2019$var_src = "LVIS 2019"
##samples_val_gliht2018_lvis2017 = data.frame(read.csv(paste(dir_TTE_map_tables, "/", "SAMPLES_stack_gliht2018_lvis2017_landforms_",tcc_type,".csv, sep=''), header=T))
samples_val_gliht2018_lvis2019 = data.frame(read.csv(paste0(maindir, "/", "SAMPLE_stack_tcc_gliht2018_tcc_lvis2019.csv"), header=T))
samples_val_gliht2018_lvis2019$var_ref = "GLiHT 2018"
samples_val_gliht2018_lvis2019$var_src = "LVIS 2019"
df_val = rbind(samples_val_gliht2014_lvis2017, samples_val_gliht2014_lvis2019, samples_val_gliht2018_lvis2019)

# Get aspect_class and landform_class from tte_classification
df_val$aspect_class = ifelse(df_val$constant < 100, 0, substr(df_val[,'constant'], 1, nchar(df_val[,'constant'])-2) )
df_val$landform_class = ifelse(df_val$constant < 100, df_val$constant, gsub("(?<![0-9])0+", "", substr(df_val[,'constant'], 2, 3 ), perl = TRUE) )

# Relabel aspect classes
df_val$aspect_class = mapvalues(as.factor(df_val$aspect_class), from = c("0","1","2","3","4"), to =  c("Flat (<3°)","North","East","South","West"))
# Indicate facet type
facet_type = "aspectclass"#"aspectclass_landformclass"
# Create the equation labels for the two groups
df_eq <- ddply(df_val %>% dplyr::select(src, ref, aspect_class, var_src, var_ref ),.(aspect_class, var_src),c("lm_eqn") )#"landform_class"

plot_lvis_gliht = ggplot(df_val, aes(x=ref, y=src))+
  labs(title = "Tree canopy cover: LVIS & GLiHT", caption = "Source: above_lidar_check.Rmd")+
    geom_bin2d(binwidth = c(1, 1))+
  labs(y="LVIS", x="GLiHT")+
  val_plot_geom_list+
  scale_y_continuous(breaks = seq(0,100,25), limits = c(0,100))+
  scale_x_continuous(breaks = seq(0,100,25), limits = c(0,100))+
  geom_label(data=df_eq, aes(x = 5, y = 94,label=paste("r^2 ==", r2,sep='')), parse = TRUE, hjust=0, vjust=0, size=2, color = "black", fill="white", label.padding=unit(0.1,"lines"))+
  geom_label(data=df_eq, aes(x = 5, y = 84,label=paste("rmse ==", rmse,sep='')), parse = TRUE, hjust=0, vjust=0, size=2, color = "black", fill="white", label.padding=unit(0.1,"lines"))+
  geom_label(data=df_eq, aes(x = 5, y = 74,label=paste("bias ==", int,sep='')), parse = TRUE, hjust=0, vjust=0, size=2, color = "black", fill="white", label.padding=unit(0.1,"lines"))+
  geom_label(data=df_eq, aes(x = 5, y = 64,label=paste("slope ==", slope,sep='')),parse = TRUE, hjust=0, vjust=0, size=2, color = "black", fill="white", label.padding=unit(0.1,"lines"))+
  #geom_text(data=df_eq, aes(x = 100, y = 5,label=paste("sig2 ==",sig2,sep='')), parse = TRUE, hjust=1, vjust=0, size=3)+
  facet_grid(var_src~aspect_class)
  #scale_fill_gradientn( labels = c(1,10,100), breaks = c(1,10,100), limits = c(1,500), colours=pal_rev_spectral9, trans = "log", name='# obs.')
print(plot_lvis_gliht)

ggplot(df_val, aes(x=ref, y=src))+
    #geom_point(alpha=0.01)+
  geom_bin2d(binwidth = c(1, 1))+

  labs(x="GLiHT tree canopy cover (%)", y="LVIS tree canopy cover (%)")+
  val_plot_geom_list+
  theme_classic()

ggsave(plot = plot_lvis_gliht,
         file = paste0("D:/projects/above_icesat2/above_lidar_chk//", "plots_TCC_validation_lvis_gliht_",tcc_type,"_",facet_type,".png"),
       device = 'png', dpi = 300, width = 10, height = 5)

In [4]:
#
# Height: Check LVIS height estimate with GLiHT
#
#
maindir = 'D:\\projects\\ABoVE_lidar'#'C:/Users/pmontesa/Google Drive/ABoVE_lidar'
RES=30
for(ref_ht_type in c('rh85','rh90','rh100')){


  if(FALSE){
    samples_val_gliht2014_lvis2017 = data.frame(read.csv(paste0(maindir, "\\", "SAMPLE_stack_",ref_ht_type,"_",RES,"m_GLIHT2014_lvisf2017_rh098_",RES,"m.csv"), header=T))
    samples_val_gliht2014_lvis2019 = data.frame(read.csv(paste0(maindir, "\\", "SAMPLE_stack_",ref_ht_type,"_",RES,"m_GLIHT2014_lvisf2019_rh098_",RES,"m.csv"), header=T))
    samples_val_gliht2018_lvis2019 = data.frame(read.csv(paste0(maindir, "\\", "SAMPLE_stack_",ref_ht_type,"_",RES,"m_GLIHT2018_lvisf2019_rh098_",RES,"m.csv"), header=T))
  }else{
    samples_val_gliht2014_lvis2017 = st_read(paste0(maindir, "\\", "SAMPLE_stack_",ref_ht_type,"_",RES,"m_GLIHT2014_lvisf2017_rh098_",RES,"m.geojson"))
    samples_val_gliht2014_lvis2019 = st_read(paste0(maindir, "\\", "SAMPLE_stack_",ref_ht_type,"_",RES,"m_GLIHT2014_lvisf2019_rh098_",RES,"m.geojson"))
    samples_val_gliht2018_lvis2019 = st_read(paste0(maindir, "\\", "SAMPLE_stack_",ref_ht_type,"_",RES,"m_GLIHT2018_lvisf2019_rh098_",RES,"m.geojson"))
  }

  ht_type = paste0("LVISFrh98_GLiHT", ref_ht_type)

  samples_val_gliht2014_lvis2017$var_ref = "GLiHT 2014"
  samples_val_gliht2014_lvis2017$var_src = "LVISF 2017"
  samples_val_gliht2014_lvis2019$var_ref = "GLiHT 2014"
  samples_val_gliht2014_lvis2019$var_src = "LVISF 2019"
  samples_val_gliht2018_lvis2019$var_ref = "GLiHT 2018"
  samples_val_gliht2018_lvis2019$var_src = "LVISF 2019"


  # Convert to basic data.frame
  df_val = as.data.frame(rbind(samples_val_gliht2014_lvis2017, samples_val_gliht2014_lvis2019, samples_val_gliht2018_lvis2019) )
  df_val$tree_fcover = df_val$tree_fcover * 100

  # Get aspect_class and landform_class from tte_classification
  df_val$aspect_class = ifelse(df_val$constant < 100, 0, substr(df_val[,'constant'], 1, nchar(df_val[,'constant'])-2) )
  df_val$landform_class = ifelse(df_val$constant < 100, df_val$constant, gsub("(?<![0-9])0+", "", substr(df_val[,'constant'], 2, 3 ), perl = TRUE) )
  cc_bin=20
  df_val$tree_fcover_class = cut(df_val$tree_fcover, seq(0,100,cc_bin),  include.lowest=TRUE, labels= as.character(seq(cc_bin,100,cc_bin)) )

  # Relabel aspect classes
  df_val$aspect_class = plyr::mapvalues(as.factor(df_val$aspect_class), from = c("0","1","2","3","4"), to =  c("Flat (<3)","North","East","South","West"))
  df_val$tree_fcover_class = plyr::mapvalues(as.factor(df_val$tree_fcover_class), from = c("20","40","60","80","100"), to =  c("< 20%","20-40%","40-60%","60-80%","80-100%"))
  # Indicate facet type
  facet_type = "aspectclass"#"aspectclass_landformclass"


  htmax = 25
  # Create the equation labels for the two groups
  plot_title = list(
    ggtitle(paste0("Boreal forest height from LVIS at ",RES,"m : comparison with legacy GLiHT"), subtitle = paste0("Crossovers of LVISF rh98 2017, 2019 with GLiHT ",ref_ht_type," 2014, 2018"))
  )
  plot_labs_scales = list(
                labs(title=NULL, subtitle=NULL, y="Canopy Height (LVISF rh98; m)",
                     # x=paste0("Canopy Height (GLiHT ",ref_ht_type,"; m)" )
                     x = as.expression(bquote("Canopy Height (GLiHT"~.(ref_ht_type)[CHM]~")"))
                      #,caption = "Source: above_lidar_check.Rmd"
                     ) ,

                scale_y_continuous(breaks = seq(0,htmax,5), limits = c(0,htmax)),
                scale_x_continuous(breaks = seq(0,htmax,5), limits = c(0,htmax))
  )
  df_eq <- ddply(df_val %>% dplyr::select(src, ref, aspect_class, var_src, var_ref ),.(aspect_class, var_src),c("lm_eqn") )#"landform_class"

  plot_lvis_gliht = ggplot(df_val, aes(x=ref, y=src))+
    #labs(title = "Height: LVIS & GLiHT", caption = "Source: above_lidar_check.Rmd")+
      geom_bin2d(binwidth = c(0.25, 0.25))+
    geom_label(data=df_eq, aes(x = 0.1, y = htmax,label=paste("r^2 ==", r2,sep='')), parse = TRUE, hjust="inward", vjust="inward", size=2, color = "black", fill="white", label.padding=unit(0.1,"lines"))+
    geom_label(data=df_eq, aes(x = 0.1, y = htmax-3,label=paste("rmse ==", rmse,sep='')), parse = TRUE, hjust="inward", vjust="inward", size=2, color = "black", fill="white", label.padding=unit(0.1,"lines"))+
    geom_label(data=df_eq, aes(x = 0.1, y = htmax-6,label=paste("bias ==", int,sep='')), parse = TRUE, hjust="inward", vjust="inward", size=2, color = "black", fill="white", label.padding=unit(0.1,"lines"))+
    geom_label(data=df_eq, aes(x = 0.1, y = htmax-9,label=paste("slope ==", slope,sep='')),parse = TRUE, hjust="inward", vjust="inward", size=2, color = "black", fill="white", label.padding=unit(0.1,"lines"))+
    #geom_text(data=df_eq, aes(x = 100, y = 5,label=paste("sig2 ==",sig2,sep='')), parse = TRUE, hjust=1, vjust=0, size=3)+
    scale_fill_gradientn( labels = c(1,10,100), breaks = c(1,10,100), limits = c(1,500), colours=pal_rev_spectral9, trans = "log", name='# obs.')+
    facet_grid(var_src+var_ref~aspect_class)+
    val_plot_geom_list+
    plot_labs_scales

  #print(plot_lvis_gliht)
  ggsave(plot = plot_lvis_gliht,
           file = paste0(maindir,"//above_lidar_chk//", "plots_HT_validation_lvis_gliht_",ht_type,"_",facet_type,"_",RES,"m.png"),
         device = 'png', dpi = 300, width = 8, height = 5)

  # Create the equation labels for the two groups
  df_eq <- ddply(df_val %>% dplyr::select(src, ref, aspect_class, var_src, var_ref, tree_fcover_class ),.(var_src, var_ref, tree_fcover_class),c("lm_eqn") )#"landform_class"
  facet_type = "year"
  plot_lvis_gliht_treefcover= ggplot(df_val, aes(x=ref, y=src))+
      #geom_point(alpha=0.01)+
    geom_bin2d(binwidth = c(.1, .1))+
    geom_label(data=df_eq, aes(x = 0.1, y = htmax, label=paste("r^2 ==", r2,sep='')), parse = TRUE, hjust="inward", vjust="inward", size=2, color = "black", fill="white", label.padding=unit(0.1,"lines"))+
    geom_label(data=df_eq, aes(x = 0.1, y = htmax-3.5, label=paste("rmse ==", rmse, sep='')), parse = TRUE, hjust="inward", vjust="inward", size=2, color = "black", fill="white", label.padding=unit(0.1,"lines"))+
    geom_label(data=df_eq, aes(x = 0.1, y = htmax-6.5, label=paste("bias ==", int, sep='')), parse = TRUE, hjust="inward", vjust="inward", size=2, color = "black", fill="white", label.padding=unit(0.1,"lines"))+
    geom_label(data=df_eq, aes(x = 0.1, y = htmax-9.5, label=paste("slope ==", slope, sep='')),parse = TRUE, hjust="inward", vjust="inward", size=2, color = "black", fill="white", label.padding=unit(0.1,"lines"))+
    scale_fill_gradientn( labels = c(1,10,100), breaks = c(1,10,100), limits = c(1,500), colours=pal_rev_spectral9, trans = "log", name='# obs.')+
    facet_grid(var_src+var_ref~tree_fcover_class)+
    val_plot_geom_list+
    plot_labs_scales

  print(plot_lvis_gliht_treefcover + plot_title)
    ggsave(plot = plot_lvis_gliht_treefcover,
           file = paste0(maindir, "//above_lidar_chk//", "plots_HT_validation_lvis_gliht_treefcover_",ht_type,"_",facet_type,"_",RES,"m.tif"),
         device = 'tiff', dpi = 300, width = 8, height = 4)
  ggsave(plot = plot_lvis_gliht_treefcover,
           file = paste0(maindir, "//above_lidar_chk//", "plots_HT_validation_lvis_gliht_treefcover_",ht_type,"_",facet_type,"_",RES,"m.png"),
         device = 'png', dpi = 300, width = 8, height = 4)
}

In [5]:
df_list = list(samples_val_gliht2014_lvis2017, samples_val_gliht2014_lvis2019, samples_val_gliht2018_lvis2019)
st_write(do.call('rbind', df_list), dsn = path(maindir, 'above_lidar_crossovers.gpkg'))

In [6]:
# Build sample df for plotting
SAMP_VAL = 0.0025
samp_df = rbind(sample_frac(samples_val_gliht2014_lvis2017, 0.01), sample_frac(samples_val_gliht2014_lvis2019, SAMP_VAL), sample_frac(samples_val_gliht2018_lvis2019, 1))
samp_df$crossover_type = paste(samp_df$var_src, " x ", samp_df$var_ref)

In [7]:
#
# Map crossover locations here?
#
# https://r-spatial.org/r/2018/10/25/ggplot2-sf-3.html

# Subset WWF according to the clusters we have SI_50 for, and join with SI_50
wwf_sf = st_as_sf(st_read('C:\\Users\\pmontesa\\Downloads\\wwf_terr_ecos_subset_regionalSI.gpkg'))
boreal = st_read("D:\\databank\\wwf\\arc\\wwf_circumboreal_Dissolve.shp") %>% st_transform(crs = crs_canalb)
above_domain = st_read("D:\\databank\\arc\\ABoVE_reference_grid_v2_1527\\ABoVE_reference_grid_v2_1527\\data\\ABoVE_Study_Domain\\ABoVE_Study_Domain\\ABoVE_Study_Domain.shp") %>% st_transform(crs = crs_canalb)

crs_wgs84 = "+proj=longlat +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0,0,0,0,0 +no_defs"

world <- ne_countries(scale = "medium", returnclass = "sf")

# https://www.r-bloggers.com/2019/04/zooming-in-on-maps-with-sf-and-ggplot2/
#disp_win_wgs84 <- st_sfc(st_point(c(-153, 54)), st_point(c(-143, 73)), crs = 4326)
disp_win_wgs84 <- st_sfc(st_point(c(-150, 57)), st_point(c(-145, 70)), crs = 4326)
disp_win_trans <- st_transform(disp_win_wgs84, crs = crs_canalb)
disp_win_coord <- st_coordinates(disp_win_trans)

# Build a df for annotation
txt_wgs84 <- st_sfc(st_point(c(-157, 63)), st_point(c(-155, 64)), st_point(c(-149, 62.5)), crs = 4326)
txt_trans <- st_transform(txt_wgs84, crs = crs_canalb)
colors = c("grey60","black","black")
sizes = c(5,5,5) # c(5,3,2)
annotation_coords <- data.frame(st_coordinates(txt_trans))
annotation_coords$label    = c("B o r e a l\nf o r e s t\nb i o m e",'','') #"ABoVE\ndomain","ABoVE\nextended\ndomain")
annotation_coords$color    = colors
annotation_coords$fontface = c("bold.italic","italic","italic")
annotation_coords$size     = sizes

if(FALSE){
  # Try to get GEOM_TEXT
  # Crop layers if need be
  # choose a point on the surface of each geometry
  above_domain_points <- sf::st_point_on_surface(above_domain %>% st_crop(extent(disp_win_coord)))
  # retrieve the coordinates
  above_domain_coords <- as.data.frame(sf::st_coordinates(above_domain_points))
  above_domain_coords$Region <- above_domain$Region
  geom_text_above_domain = list(geom_text(data = above_domain_coords, aes(X, Y, label = Region), colour = "white")) # points want to plot in XY space not in crs space; need to convert back to st_as_sf?
}



p_map_crossovers = do_map_above_boreal(ABOVE_DOMAIN=FALSE) +
  # geom_sf(data = wwf_sf, color='black',
  #           #fill="#a6dba0",
  #           fill = 'grey50',
  #           linetype = 'dotted', show.legend = TRUE, size=0.25) +
    geom_sf(data=samp_df, aes(color=crossover_type,fill=crossover_type), size=0.5)+
    #geom_sf(data=disp_win_wgs84, size=15)+
    #scale_color_manual(values = wesanderson::wes_palette("Zissou1",n = 5)[seq(1,5,2)], name = "Crossover Type", guide=F)+
    #scale_fill_manual(values = wesanderson::wes_palette("Zissou1",n = 5)[seq(1,5,2)], name = 'Crossover Type')+
    scale_color_manual(values = wesanderson::wes_palette("Darjeeling1",n = 3), name = "Crossover Type", guide=F)+
    scale_fill_manual(values = wesanderson::wes_palette("Darjeeling1",n = 3), name = 'Crossover Type')+
    theme_bw()+
    theme(
        #legend.position = c(0.8 ,0.85),
      legend.position = c(0.8 , 0.85),
        legend.text=element_text(size=rel(1)),
        legend.background = element_blank(),
        legend.box.background = element_rect(colour = "black"),
        axis.title.x = element_blank(),
        axis.title.y = element_blank()) +
  coord_sf(xlim = disp_win_coord[,'X'], ylim = disp_win_coord[,'Y'], crs = crs_canalb, expand = F)+
  annotation_scale(location = "bl", width_hint = 0.15)+
  #annotate(geom = "text", x = txt_coord[,'X'], y = txt_coord[,'Y'], label = c("B o r e a l","ABoVE\ndomain","ABoVE\nextended\ndomain"), fontface = c("bold.italic","italic","italic"), color = c("grey60","black","black"), size = c(5,3,2)) +
  geom_text(data = annotation_coords, aes(x = X, y = Y, label = label, fontface = fontface), size = sizes, color=colors) +
  #geom_sf_label(data = wwf_sf, aes(label = clusterid), size=2, fill='NA') +
  guides( color = guide_legend(override.aes = list(size = 4)))

print(p_map_crossovers)

ggsave(plot = p_map_crossovers,
           file = paste0(maindir, "//above_lidar_chk//", "plot_map_crossovers.tif"),
         device = 'tiff', dpi = 150, width = 8, height = 5)
ggsave(plot = p_map_crossovers,
           file = paste0(maindir, "//above_lidar_chk//", "plot_map_crossovers.png"),
         device = 'png', dpi = 150, width = 8, height = 6)